In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import pymongo
from pymongo import MongoClient

In [2]:
# Connection to mongo
client = MongoClient('mongodb+srv://<username>:<password>@cluster0.l3pqt.mongodb.net/MSA?retryWrites=true&w=majority')
# Select database
db = client['MSA']
# see list of collections
client.MSA.list_collection_names()

['arima_LaborParticipation_pred_score',
 'CPI_RegionUrban_raw',
 'BusinessApplications_clean',
 'arima_GDP_2019_2024',
 'employment_prediction',
 'Employment_clean',
 'CPI_RegionUrban_clean',
 'population_predicted_2024',
 'employment_clean',
 'LaborForce_clean',
 'GDP_raw',
 'unemployment_ROC_2024',
 'employment_raw',
 'OES_raw',
 'Population_clean',
 'unemployment_predicted_2024',
 'arima_pop_pred_score',
 'msa_codes',
 'arima_unem_pred_score',
 'population_raw',
 'arima_emp_pred_score',
 'LaborForce_raw',
 'arima_cpi_pred_score',
 'UnemploymentRate_clean',
 'Employment_raw',
 'arima_2024_ROC_score_total',
 'regions_divisions',
 'LaborParticipation_clean',
 'unemployment_raw',
 'AnnualMeanWage_clean',
 'GDP_clean',
 'UnemploymentRate_raw',
 'Predicted_2024_ROC_rank_total',
 'arima_GDP_pred_score',
 'Predicted_2024_ROC_rank_total2',
 'GDP_predicted_2024']

In [3]:
# Select the collection with needed data
unemp = db.UnemploymentRate_clean
df = pd.DataFrame(list(unemp.find()))
print(df.dtypes)
df

_id      object
CBSA      int64
2010    float64
2011    float64
2012    float64
2013    float64
2014    float64
2015    float64
2016    float64
2017    float64
2018    float64
2019    float64
dtype: object


,_id,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,603dbf8d680af8db371b2136,10180,6.6,5.9,5.2,4.5,3.4,3.5,3.8,3.2,3.1,2.7
1,603dbf8d680af8db371b2137,10420,9.6,7.9,7.4,6.7,5.1,4.9,5.3,4.7,4.6,4.0
2,603dbf8d680af8db371b2138,10500,11.6,10.8,10.0,8.9,7.2,6.4,5.8,4.8,4.8,3.6
3,603dbf8d680af8db371b2139,10540,12.0,11.0,10.6,8.6,6.9,5.9,4.6,4.4,4.5,3.4
4,603dbf8d680af8db371b213a,10580,7.1,7.1,6.9,5.3,4.4,4.0,4.0,4.1,3.5,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...
379,603dbf8d680af8db371b22b1,49420,12.0,12.7,11.9,11.5,10.7,10.6,8.9,8.1,8.6,8.0
380,603dbf8d680af8db371b22b2,49620,7.3,7.0,6.9,5.5,4.5,4.0,4.0,3.7,3.3,3.8
381,603dbf8d680af8db371b22b3,49660,10.4,8.6,8.6,7.4,5.7,7.2,6.2,5.6,5.6,5.4
382,603dbf8d680af8db371b22b4,49700,19.1,17.5,15.8,13.4,11.4,10.2,9.8,7.9,7.8,6.5


In [4]:
df.drop(columns='_id', inplace=True)
df.head()

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,6.6,5.9,5.2,4.5,3.4,3.5,3.8,3.2,3.1,2.7
1,10420,9.6,7.9,7.4,6.7,5.1,4.9,5.3,4.7,4.6,4.0
2,10500,11.6,10.8,10.0,8.9,7.2,6.4,5.8,4.8,4.8,3.6
3,10540,12.0,11.0,10.6,8.6,6.9,5.9,4.6,4.4,4.5,3.4
4,10580,7.1,7.1,6.9,5.3,4.4,4.0,4.0,4.1,3.5,3.7


In [5]:
# for loop to predict 2019 so percent error score can be calculated
predictions = []
for i in range(0,384):
    y = df.iloc[i, 1:-1].values
    series = pd.Series(y, dtype='float')
    model = ARIMA(series, order=(2, 1, 1))
    model_fit = model.fit()
    pred = model_fit.forecast(1)
    predictions.append(pred.values.tolist()[0])
df['2019_pred'] = predictions

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: Convergen

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

In [6]:
df.head()

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2019_pred
0,10180,6.6,5.9,5.2,4.5,3.4,3.5,3.8,3.2,3.1,2.7,3.218783
1,10420,9.6,7.9,7.4,6.7,5.1,4.9,5.3,4.7,4.6,4.0,4.984974
2,10500,11.6,10.8,10.0,8.9,7.2,6.4,5.8,4.8,4.8,3.6,4.473119
3,10540,12.0,11.0,10.6,8.6,6.9,5.9,4.6,4.4,4.5,3.4,4.502729
4,10580,7.1,7.1,6.9,5.3,4.4,4.0,4.0,4.1,3.5,3.7,3.123946


In [7]:
# calculate percent error for each linear regression
df['Percent_Error'] = (df['2019'] - df['2019_pred'])/df['2019']*100

# set values to absolute in percent_error column
df['Percent_Error'] = df['Percent_Error'].abs()

# print mean of a percent_error
mean_percent_error = df['Percent_Error'].mean()
print(f'Average percent error for ARIMA predicting 2019 unempoloyment rate is: {round(mean_percent_error,3)}%' )
df.head()

Average percent error for ARIMA predicting 2019 unempoloyment rate is: 16.07%


,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2019_pred,Percent_Error
0,10180,6.6,5.9,5.2,4.5,3.4,3.5,3.8,3.2,3.1,2.7,3.218783,19.214175
1,10420,9.6,7.9,7.4,6.7,5.1,4.9,5.3,4.7,4.6,4.0,4.984974,24.624356
2,10500,11.6,10.8,10.0,8.9,7.2,6.4,5.8,4.8,4.8,3.6,4.473119,24.253319
3,10540,12.0,11.0,10.6,8.6,6.9,5.9,4.6,4.4,4.5,3.4,4.502729,32.433214
4,10580,7.1,7.1,6.9,5.3,4.4,4.0,4.0,4.1,3.5,3.7,3.123946,15.569032
